# Layout optimization

This example demonstrates some basics about running wind farm optimization tasks with `foxes`. All optimizations use the [iwopy](https://github.com/FraunhoferIWES/iwopy) interface in the background (also by Fraunhofer IWES, see link for details). Let's first make sure this and other required packages are installed, as a standard user by 
```console
pip install foxes[opt]
pip install pymoo
```
Note that if you have installed `foxes` as a developer you may replace the first line by `pip install iwopy`. The second line installs the optimization library [pymoo](https://pymoo.org/) which contains a number of very nice genetic algorithm implementations. We are installing this here because we are going to ask `iwopy` to use one of them later on.

These are the required imports for this example:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from plotly.offline import iplot

import foxes
import foxes.variables as FV
import foxes.utils.geom2d as gm

In the following we are tackling the problem of optimizing a wind farm layout for a site near Bremen, Germany. The data of a (coarse) wind rose with 216 states is provided as static data file with name `"wind_rose_bremen.csv"`:
```
state,wd,ws,weight
0,0.0,3.5,0.00158
1,0.0,6.0,0.00244
2,0.0,8.5,0.00319
3,0.0,12.5,0.0036700002
4,0.0,17.5,0.00042
...
```
First, let's create the states object and have a look at the wind rose:

In [ ]:
states = foxes.input.states.StatesTable(
    data_source="wind_rose_bremen.csv",
    output_vars=[FV.WS, FV.WD, FV.TI, FV.RHO],
    var2col={FV.WS: "ws", FV.WD: "wd", FV.WEIGHT: "weight"},
    fixed_vars={FV.RHO: 1.225, FV.TI: 0.05},
)

o = foxes.output.AmbientRosePlotOutput(states, point=[0., 0., 100.])
fig = o.get_figure(16, FV.AMB_WS, [0, 3.5, 6, 10, 15, 20])
iplot(fig)

Next, we need to specify the area within which the turbines are allowed to move during optimization. We use the `foxes.utils.geom2d` sub-package for that purpose (imported as `gm`, see above) which allows us to add and subtract polygons, circles, etc.

In [ ]:
boundary = \
    gm.ClosedPolygon(np.array(
    [[0, 0], [0, 1200], [1000, 800], [900, -200]], dtype=np.float64)) \
    + gm.ClosedPolygon(np.array(
    [[500, 0], [500, 1500], [1000, 1500], [1000, 0]], dtype=np.float64)) \
    - gm.Circle([-100., -100.], 700)

fig, ax = plt.subplots(figsize=(5, 8))
boundary.add_to_figure(ax)
plt.show()

Later on we wish to apply boundary constraints that make sure all turbines are placed within this area geometry. These conditions make use of the minimal distance calculation from each point in question to the boundary. We can check the results by plotting again, now using the `fill_mode` option:

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(14, 8))
boundary.add_to_figure(axs[0], fill_mode="dist_inside")
boundary.add_to_figure(axs[1], fill_mode="dist_outside")
plt.show()